In [186]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

#mpl.rcParams["figure.facecolor"]
#mpl.rcParams["axes.facecolor"]
mpl.rcParams["savefig.facecolor"]='FFFFFF'

for year in np.arange(2011,2022):
    df = pd.read_excel(f'data/timvarden-{year}-01-12.xls', skiprows=4, converters= {f'{year}': pd.to_datetime})
    df = df.rename(columns=lambda x: x.lstrip())
    
    #df.head()
    print(df.keys())

    #sol
    df['SolkraftproduktionTotMWh'] = df['SolkraftproduktionSE4MWh'] + df['SolkraftproduktionSE3MWh'] + \
        df['SolkraftproduktionSE2MWh'] + df['SolkraftproduktionSE1MWh']

    df['SolkraftproduktionTotGWh'] = df['SolkraftproduktionTotMWh'] / 1e3

    #vind
    df['VindkraftproduktionTotMWh'] = df['VindkraftproduktionSE1MWh'] + df['VindkraftproduktionSE2MWh'] + \
        df['VindkraftproduktionSE3MWh'] + df['VindkraftproduktionSE4MWh']

    df['VindkraftproduktionTotGWh'] = df['VindkraftproduktionTotMWh'] / 1e3

    #vatten
    df['Vattenkraft produktionTotMWh'] = df['Vattenkraft produktionSE1MWh'] + df['Vattenkraft produktionSE2MWh'] + \
        df['Vattenkraft produktionSE3MWh'] + df['Vattenkraft produktionSE4MWh']

    df['Vattenkraft produktionTotGWh'] = df['Vattenkraft produktionTotMWh'] / 1e3

    #värme
    df['VärmekraftproduktionTotMWh'] = df['VärmekraftproduktionSE1MWh'] + df['VärmekraftproduktionSE2MWh'] + \
        df['VärmekraftproduktionSE3MWh'] + df['VärmekraftproduktionSE4MWh']

    df['VärmekraftproduktionTotGWh'] = df['VärmekraftproduktionTotMWh'] / 1e3

    #kärn
    df['KärnkraftproduktionTotMWh'] = df['KärnkraftproduktionSE3MWh']

    df['KärnkraftproduktionTotGWh'] = df['KärnkraftproduktionTotMWh'] / 1e3

    #ospec
    if year < 2016:
        #create empty frame for data missing from xls.
        df['Ospec. produktionSE1MWh'] = df['KärnkraftproduktionTotGWh'] * 0
    
    df['OspecproduktionTotMWh'] = df['Ospec. produktionSE1MWh'] + df['Ospec. produktionSE2MWh'] + \
        df['Ospec. produktionSE3MWh'] + df['Ospec. produktionSE4MWh']

    df['OspecproduktionTotGWh'] = df['OspecproduktionTotMWh'] / 1e3

    #gasturbin diesel
    #from 2018 this data is gone
    #so we fill it with empty
    if year > 2017:
        for SE in [1, 2, 3, 4]:
            df[f'Gast./dieselproduktionSE{SE}MWh'] = df['KärnkraftproduktionTotGWh']*0
    
    df['Gast./dieselproduktionTotMWh'] = df['Gast./dieselproduktionSE1MWh'] + \
        df['Gast./dieselproduktionSE2MWh'] + df['Gast./dieselproduktionSE3MWh'] +\
        df['Gast./dieselproduktionSE4MWh']
    df['Gast./dieselproduktionTotGWh'] = df['Gast./dieselproduktionTotMWh'] / 1e3
    
    #create consumption and production sums for each SE
    for SE in [1, 2, 3, 4]:
        
        if year < 2019:
            #create empty frames if it doesn't exist
            df[f'Timmätt förbr>50 MWSE{SE}MWh'] = df[f'Timmätt förbrexkl. avk.lastSE{SE}MWh'] * 0
            df[f'TimmättaförlusterSE{SE}MWh'] = df[f'Timmätt förbrexkl. avk.lastSE{SE}MWh'] * 0
        
        df[f'nettkonsumptionSE{SE}GWh'] = -df[f'Timmätt förbrexkl. avk.lastSE{SE}MWh']/1e3 +\
            -df[f'Timmätt förbr>50 MWSE{SE}MWh']/1e3 + -df[f'Avkopplingsb.lastSE{SE}MWh']/1e3 + \
            -df[f'TimmättaförlusterSE{SE}MWh']/1e3

        if SE == 3:
            df[f'nettoproduktionSE{SE}GWh'] = df[f'Vattenkraft produktionSE{SE}MWh']/1e3 + \
                    df[f'VindkraftproduktionSE{SE}MWh']/1e3 + df[f'KärnkraftproduktionSE{SE}MWh']/1e3 + \
                    df[f'VärmekraftproduktionSE{SE}MWh']/1e3 + df[f'SolkraftproduktionSE{SE}MWh']/1e3 + \
                    df[f'Ospec. produktionSE{SE}MWh']/1e3
        else:
            df[f'nettoproduktionSE{SE}GWh'] = df[f'Vattenkraft produktionSE{SE}MWh']/1e3 + \
                    df[f'VindkraftproduktionSE{SE}MWh']/1e3 + df['KärnkraftproduktionSE3MWh']*0/1e3 + \
                    df[f'VärmekraftproduktionSE{SE}MWh']/1e3 + df[f'SolkraftproduktionSE{SE}MWh']/1e3 + \
                    df[f'Ospec. produktionSE{SE}MWh']/1e3 + df[f'Gast./dieselproduktionSE{SE}MWh']/1e3


    #fix time

    df['time'] = df[f'{year}']
    df['angles'] = np.linspace(0, np.pi*2, len(df[f'{year}']))
    df['xticks'] = np.linspace(1, 13, len(df[f'{year}']))
    #df.head()

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)

    plt.plot(df['angles'], df['Vattenkraft produktionTotGWh'], label='Vatten', alpha=0.8)
    plt.plot(df['angles'], df['VindkraftproduktionTotGWh'], label='Vind', alpha=0.8)
    plt.plot(df['angles'], df['KärnkraftproduktionTotGWh'], label='Fission', alpha=0.8, linewidth=2)
    plt.plot(df['angles'], df['VärmekraftproduktionTotGWh'], label='Förbr', alpha=0.8)
    plt.plot(df['angles'], df['SolkraftproduktionTotGWh'], label='Sol', alpha=0.8)
    plt.plot(df['angles'], df['OspecproduktionTotGWh'], label='Ospec', alpha=0.8)
    plt.plot(df['angles'], df['Gast./dieselproduktionTotGWh'], label='Gast./Diesel', alpha=0.8)
    
    #radialtick
    ax.set_rticks([0, 5, 10])
    ax.set_rlabel_position(-55)  # Move radial labels away from plotted line

    #thetatick varje månad
    ax.set_xticks(np.arange(0,2.0*np.pi,2*np.pi/12.))
    ax.set_xticklabels(['jan', 'feb', 'mar', 'apr', 'maj', 'jun', 'jul', 'aug', 'sept', 'okt', 'nov', 'dec'])

    angle = np.deg2rad(12)
    ax.legend(loc="lower left",
              bbox_to_anchor=(.6 + np.cos(angle)/2, .6 + np.sin(angle)/2))

    #titel och annot
    fig.suptitle(f'Timvis elkraftproduktion {year} [GWh]', fontsize=10)

    plt.text(0.1, -0.001, 'https://github.com/pfyhr/elkraftplot', fontsize=10, transform=plt.gcf().transFigure)

    plt.savefig(f'plots/polar/kraftsystem_pol_{year}.png', dpi=600, bbox_inches='tight')
    plt.close()

    #vanlig plot

    fig, ax = plt.subplots()


    plt.plot(df['xticks'], df['Vattenkraft produktionTotGWh'], label='Vatten', alpha=0.8)
    plt.plot(df['xticks'], df['VindkraftproduktionTotGWh'], label='Vind', alpha=0.8)
    plt.plot(df['xticks'], df['KärnkraftproduktionTotGWh'], label='Fission', alpha=0.8, linewidth=2)
    plt.plot(df['xticks'], df['VärmekraftproduktionTotGWh'], label='Förbr', alpha=0.8)
    plt.plot(df['xticks'], df['SolkraftproduktionTotGWh'], label='Sol', alpha=0.8)
    plt.plot(df['xticks'], df['OspecproduktionTotGWh'], label='Ospec', alpha=0.8)
    plt.plot(df['xticks'], df['Gast./dieselproduktionTotGWh'], label='Gast./Diesel', alpha=0.8)


    ax.set_xticks(np.arange(1,13,1))
    ax.set_xticklabels(['jan', 'feb', 'mar', 'apr', 'maj', 'jun', 'jul', 'aug', 'sept', 'okt', 'nov', 'dec'])
    plt.title(f'Timvis elkraftproduktion {year}', fontsize=10)
    ax.legend(bbox_to_anchor=(1.02, 0.75))
    ax.set_ylabel('Elkraftproduktion [GWh]')

    plt.text(0.1, -0.001, 'https://github.com/pfyhr/elkraftplot', fontsize=10, transform=plt.gcf().transFigure)

    plt.savefig(f'plots/line/kraftsystem_norm_{year}.png', dpi=600, bbox_inches='tight')
    plt.close()

    fig, ax = plt.subplots()

    labels = ['Vatten', 'Vind', 'Fission', 'Förbr', 'Sol', 'Ospec', 'Gast./Diesel']
    plt.stackplot(df['xticks'], df['Vattenkraft produktionTotGWh'], \
                    df['VindkraftproduktionTotGWh'], df['KärnkraftproduktionTotGWh'], \
                    df['VärmekraftproduktionTotGWh'], df['SolkraftproduktionTotGWh'], \
                    df['OspecproduktionTotGWh'], df['Gast./dieselproduktionTotGWh'], labels=labels, alpha=0.9)

    ax.set_xticks(np.arange(1,13,1))
    ax.set_xticklabels(['jan', 'feb', 'mar', 'apr', 'maj', 'jun', 'jul', 'aug', 'sept', 'okt', 'nov', 'dec'])
    plt.title('Timvis elkraftproduktion i Sverige', fontsize=10)
    ax.legend(bbox_to_anchor=(1.25, 0.75))
    ax.set_ylabel(f'Timvis elkraftproduktion {year} [GWh]')

    plt.text(0.1, -0.001, 'https://github.com/pfyhr/elkraftplot', fontsize=10, transform=plt.gcf().transFigure)

    plt.savefig(f'plots/stack/kraftsystem_stack_{year}.png', dpi=600, bbox_inches='tight')
    plt.close()

    fig, axs = plt.subplots(2,2, sharex=True, sharey=True)
    plt.tight_layout()

    labels = ['Vatten', 'Vind', 'Fission', 'Värmekr.', 'Sol', 'Ospec', 'Gast./Diesel']

    for SE in [1, 2, 3, 4]:
        if SE == 1:
            row = 0
            col = 0
        elif SE == 2:
            row = 0
            col = 1
        elif SE == 3:
            row = 1
            col = 0
        else:
            row = 1
            col = 1

        if SE == 3:
            axs[row,col].stackplot(df['xticks'], df[f'Vattenkraft produktionSE{SE}MWh']/1e3, \
                    df[f'VindkraftproduktionSE{SE}MWh']/1e3, df[f'KärnkraftproduktionSE{SE}MWh']/1e3, \
                    df[f'VärmekraftproduktionSE{SE}MWh']/1e3, df[f'SolkraftproduktionSE{SE}MWh']/1e3, \
                    df[f'Ospec. produktionSE{SE}MWh']/1e3, df[f'Gast./dieselproduktionSE{SE}MWh']/1e3, \
                                   labels=labels, alpha=0.9)
        else:
            axs[row,col].stackplot(df['xticks'], df[f'Vattenkraft produktionSE{SE}MWh']/1e3, \
                    df[f'VindkraftproduktionSE{SE}MWh']/1e3, df['KärnkraftproduktionSE3MWh']*0/1e3, \
                    df[f'VärmekraftproduktionSE{SE}MWh']/1e3, df[f'SolkraftproduktionSE{SE}MWh']/1e3, \
                    df[f'Ospec. produktionSE{SE}MWh']/1e3, df[f'Gast./dieselproduktionSE{SE}MWh']/1e3, \
                                   labels=labels, alpha=0.9)


        if row == 1:
            axs[row,col].set_xticks(np.arange(1,13,1))
            axs[row,col].set_xticklabels( \
             ['jan', 'feb', 'mar', 'apr', 'maj', 'jun', 'jul', 'aug', 'sept', 'okt', 'nov', 'dec'] \
             , rotation = 55)

        if col == 0:
            axs[row,col].set_ylabel('Timvis prod. [GWh]')
        axs[row,col].set_title(f'Elkraftproduktion i SE{SE} {year}', fontsize=10)

    #add the legend
    axs[row,col].legend(bbox_to_anchor=(1.55, 1.5))

    #grab the yscale for the next plot
    ylims = axs[row,col].get_ylim()

    plt.text(0.1, -0.04, 'https://github.com/pfyhr/elkraftplot', fontsize=10, transform=plt.gcf().transFigure)
    plt.savefig(f'plots/stack_SE_prod/kraftsystem_stack_SEs_{year}.png', dpi=600, bbox_inches='tight')
    plt.close()

    fig, axs = plt.subplots(2,2, sharex=True, sharey=True)
    plt.tight_layout()

    labels = ['Ex avk. last', '>50 MW', 'Avkoppl.b.', 'Förluster']

    for SE in [1, 2, 3, 4]:
        if SE == 1:
            row = 0
            col = 0
        elif SE == 2:
            row = 0
            col = 1
        elif SE == 3:
            row = 1
            col = 0
        else:
            row = 1
            col = 1


        axs[row,col].stackplot(df['xticks'], -df[f'Timmätt förbrexkl. avk.lastSE{SE}MWh']/1e3, \
            -df[f'Timmätt förbr>50 MWSE{SE}MWh']/1e3, -df[f'Avkopplingsb.lastSE{SE}MWh']/1e3, \
            -df[f'TimmättaförlusterSE{SE}MWh']/1e3, labels=labels, alpha=0.9)

        if row == 1:
            axs[row,col].set_xticks(np.arange(1,13,1))
            axs[row,col].set_xticklabels( \
             ['jan', 'feb', 'mar', 'apr', 'maj', 'jun', 'jul', 'aug', 'sept', 'okt', 'nov', 'dec'] \
             , rotation = 55)
        if col == 0:
            axs[row,col].set_ylabel('Timvis kons. [GWh]')    
        axs[row,col].set_title(f'Elkraftkonsumption i SE{SE} {year}', fontsize=10)


    axs[row,col].set_ylim(ylims)

    #add the legend
    axs[row,col].legend(bbox_to_anchor=(1.60, 1.45))

    plt.text(0.1, -0.04, 'https://github.com/pfyhr/elkraftplot', fontsize=10, transform=plt.gcf().transFigure)
    plt.savefig(f'plots/stack_SE_cons/forbrukning_stack_SEs_{year}.png', dpi=600, bbox_inches='tight')
    plt.close()

    fig, axs = plt.subplots(2,2, sharex=True, sharey=True)
    plt.tight_layout()

    labels = ['netto =\nprod.-kons.']

    for SE in [1, 2, 3, 4]:
        if SE == 1:
            row = 0
            col = 0
        elif SE == 2:
            row = 0
            col = 1
        elif SE == 3:
            row = 1
            col = 0
        else:
            row = 1
            col = 1


        axs[row,col].stackplot(df['xticks'], df[f'nettoproduktionSE{SE}GWh'] - df[f'nettkonsumptionSE{SE}GWh'], \
           labels=labels, alpha=0.9)

        if row == 1:
            axs[row,col].set_xticks(np.arange(1,13,1))
            axs[row,col].set_xticklabels( \
             ['jan', 'feb', 'mar', 'apr', 'maj', 'jun', 'jul', 'aug', 'sept', 'okt', 'nov', 'dec'] \
             , rotation = 55)
        if col == 0:
            axs[row,col].set_ylabel('Timvis netto. [GWh]')    
        axs[row,col].set_title(f'Elkraftnetto i SE{SE} {year}', fontsize=10)

    offset = 5
    netylims = (ylims[0]-offset, ylims[1]-offset)
    axs[row,col].set_ylim(netylims)

    #add the legend
    axs[row,col].legend(bbox_to_anchor=(1.60, 1.25))
    plt.text(0.1, -0.04, 'https://github.com/pfyhr/elkraftplot', fontsize=10, transform=plt.gcf().transFigure)

    plt.savefig(f'plots/stack_SE_net/netto_stack_SEs_{year}.png', dpi=600, bbox_inches='tight')
    plt.close()


Index(['2011', 'Timmätt förbrexkl. avk.lastSE1MWh',
       'Timmätt förbrexkl. avk.lastSE2MWh',
       'Timmätt förbrexkl. avk.lastSE3MWh',
       'Timmätt förbrexkl. avk.lastSE4MWh', 'Ospec.förbrukningSE2MWh',
       'Ospec.förbrukningSE3MWh', 'Ospec.förbrukningSE4MWh',
       'Avkopplingsb.lastSE1MWh', 'Avkopplingsb.lastSE2MWh',
       'Avkopplingsb.lastSE3MWh', 'Avkopplingsb.lastSE4MWh',
       'Ospec. produktionSE2MWh', 'Ospec. produktionSE3MWh',
       'Ospec. produktionSE4MWh', 'Vattenkraft produktionSE1MWh',
       'Vattenkraft produktionSE2MWh', 'Vattenkraft produktionSE3MWh',
       'Vattenkraft produktionSE4MWh', 'VindkraftproduktionSE1MWh',
       'VindkraftproduktionSE2MWh', 'VindkraftproduktionSE3MWh',
       'VindkraftproduktionSE4MWh', 'KärnkraftproduktionSE3MWh',
       'VärmekraftproduktionSE1MWh', 'VärmekraftproduktionSE2MWh',
       'VärmekraftproduktionSE3MWh', 'VärmekraftproduktionSE4MWh',
       'Gast./dieselproduktionSE1MWh', 'Gast./dieselproduktionSE2MWh',
     

Index(['2017', 'Timmätt förbrexkl. avk.lastSE1MWh',
       'Timmätt förbrexkl. avk.lastSE2MWh',
       'Timmätt förbrexkl. avk.lastSE3MWh',
       'Timmätt förbrexkl. avk.lastSE4MWh', 'Ospec.förbrukningSE1MWh',
       'Ospec.förbrukningSE2MWh', 'Ospec.förbrukningSE3MWh',
       'Ospec.förbrukningSE4MWh', 'Avkopplingsb.lastSE1MWh',
       'Avkopplingsb.lastSE2MWh', 'Avkopplingsb.lastSE3MWh',
       'Avkopplingsb.lastSE4MWh', 'Ospec. produktionSE1MWh',
       'Ospec. produktionSE2MWh', 'Ospec. produktionSE3MWh',
       'Ospec. produktionSE4MWh', 'Vattenkraft produktionSE1MWh',
       'Vattenkraft produktionSE2MWh', 'Vattenkraft produktionSE3MWh',
       'Vattenkraft produktionSE4MWh', 'VindkraftproduktionSE1MWh',
       'VindkraftproduktionSE2MWh', 'VindkraftproduktionSE3MWh',
       'VindkraftproduktionSE4MWh', 'KärnkraftproduktionSE3MWh',
       'VärmekraftproduktionSE1MWh', 'VärmekraftproduktionSE2MWh',
       'VärmekraftproduktionSE3MWh', 'VärmekraftproduktionSE4MWh',
       'Gast./d